# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Jose Naranjo  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>

Descripción del problema:(copiar enunciado)

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>
¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?<br>

Respuesta

Si no tomamos en cuenta las restricciones, cada toma podría asignarse a cualquier día. Esto teóricamente nos permite un número infinito de posibilidades, a menos que definamos un número máximo de días. Si creamos una restricción con respecto al número de días, entonces la cantidad de posibilidades estará limitada al número de días.

Si consideramos restricciones, sabemos que tendremos menos posibilidades que si no consideramos restricciones. Como los actores participan en distintas tomas, tenemos que analizar la matriz para determinar qué tomas pueden agruparse en un mismo día para no violar las restricciones. Resolver esto implica realizar una optimización que minimice el número total de días, y así hallar el conjunto de asignaciones de tomas a días.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Para desarrollar el reto se nos proporcionó una matriz de adyacencia entre tomas y actores (30x10) donde 1 representa que el actor participa en la toma y 0 que no. Esta es la estructura inicial que se adapta intuitivamente al problema. Sin embargo, investigué que también puede ser útil usar una representación por grafos. Durante el desarrollo de los algoritmos podré evaluar si es que tiene más sentido una estructura versus otra.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo es minimizar el costo total asociado a la contratación de los actores. Como los actores cobran una cantidad fija por día, independientemente del número de tomas que realicen en el día, el objetivo se puede representar como minimizar el número de días necesarios para completar todas las tomas.

Se puede expresar como: Minimizar F(x) = número de días

Donde x representa una asignación de tomas a días que cumpla con las restricciones:
- Cada actor debe estar presente en todas las tomas de su personaje
- No se pueden grabar más de 6 tomas al día

Es un problema de minimización, donde se busca reducir el número de días, lo cual a su vez minimiza el costo de los actores (que cobran por día).

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [ ]:
import pandas as pd
from itertools import combinations, chain, product

df = pd.read_csv("matriz.csv")
df.set_index('Toma', inplace=True)
matriz = df.to_numpy()

max_tomas_por_dia = 6
total_tomas = len(matriz)
tomas = set(range(1, total_tomas + 1))

def verificar_restriccion(combinacion):
    for i in range(len(combinacion)):
        for j in range(i+1, len(combinacion)):
            # si 2 tomas comparten un actor, no es valido
            if any(matriz[combinacion[i]-1][k] and matriz[combinacion[j]-1][k] for k in range(len(matriz[0]))):
                return False
    return True

def fuerza_bruta():
    tomas = range(1, total_tomas + 1)
    combinaciones_validas = []

    # genera todas las combinaciones posibles que cumplan la restriccion
    for r in range(1, max_tomas_por_dia + 1):
        for combinacion in combinations(tomas, r):
            if verificar_restriccion(combinacion):
                combinaciones_validas.append(combinacion)
    
    return combinaciones_validas

def encontrar_todas_los_dias(combinaciones_validas):
    # encuentra todas las formas posibles de cubrir todas las tomas
    # con las combinaciones validas
    for numero_dias in range(1, len(combinaciones_validas) + 1):
        for dias in combinations(combinaciones_validas, numero_dias):
            todas_las_tomas = set(chain.from_iterable(dias))
            if todas_las_tomas == tomas:
                yield dias

def minimizar(combinaciones_validas):
    solucion_optima = None
    numero_minimo_dias = float('inf') # iniciamos con infinito
    
    for posible_solucion in encontrar_todas_los_dias(combinaciones_validas):
        if len(posible_solucion) < numero_minimo_dias:
            solucion_optima = posible_solucion
            numero_minimo_dias = len(posible_solucion)
    
    return solucion_optima

combinaciones_validas = fuerza_bruta()
solucion = minimizar(combinaciones_validas)
print(solucion)

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

El algoritmo se hace en 2 etapas. En la primera, generamos todas las combinaciones válidas por día, respetando la restricción de que no más de 6 tomas pueden ser asignadas a un día y que las tomas que involucran a los mismos actores no pueden ser el mismo día.

Luego encontramos todas las posibles formas de agrupar estas combinaciones para cubrir todas las tomas con el menor número de días, así reduciendo el costo. Esto se hace seleccionando las combinaciones válidas que juntas cubren el conjunto total de tomas. Este segundo paso es computacionalmente muy complejo, dado que necesitamos considerar todas las sub-combinaciones posibles de las combinaciones válidas. Esto escala exponencialmente con el número de combinaciones.

Entonces la complejidad es 2^C donde C es el número de combinaciones válidas. Lo cual hace que sea un interesante ejercicio teórico pero no práctico en la vida real. De hecho, no pude terminar de ejecutar el algoritmo sin que mi procesador se rinda.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [28]:
# Un algoritmo que mejora la complejidad del algoritmo por fuerza bruta puede ser un algoritmo genético.
# Este es un algoritmo metaheurístico que por definición nos puede llevar a una solución razonable con
# bajos recursos, que es el caso que estamos buscando realizar aquí. Adicionalmente, consideré también
# algoritmos voraces, de programación entera (aunque no tengo disponible el software para hacerlo)
# y búsqueda local. Me parece que algoritmos genéticos tienen una implementación interesante que me gustaría
# seguir aprendiendo y por eso lo elegí.

# La complejidad de un algoritmo genético depende de muchos factores porque ocurren muchas operaciones
# dentro del algoritmo. Sin embargo, debería ser más eficiente que fuerza bruta por definición.
# Por otro lado, luego de realizar pruebas, para empezar sí logro terminar de ejecutar el algoritmo y tener
# una solución razonable, así que al menos en términos de tiempo de cómputo, es más eficiente.

# A continuación mi implementación de un algoritmo genético para resolver el problema:

import numpy as np
import random

def generar_individuo(tomas, dias):
    # genera un individuo aleatorio
    return [random.randint(1, dias) for _ in range(tomas)]

def calcular_fitness(individuo, matriz, max_tomas_por_dia):
    # calcula el fitness de un individuo usando la restriccion de max_tomas_por_dia
    dias_usados = set(individuo)
    fitness = 0
    
    # penalización por no cumplir restricciones
    for dia in dias_usados:
        tomas_en_dia = [i for i, x in enumerate(individuo) if x == dia]
        if len(tomas_en_dia) > max_tomas_por_dia:
            fitness -= 1  # penaliza por exceder el número de tomas por día
            
        # verifica actores en las tomas del mismo día
        for i in range(len(tomas_en_dia)):
            for j in range(i + 1, len(tomas_en_dia)):
                if np.any(matriz[tomas_en_dia[i]] & matriz[tomas_en_dia[j]]):
                    fitness += 1
    return fitness

def escoger_padres(poblacion, matriz, max_tomas_por_dia):
    # escoge padres segun fitness
    fitness = [calcular_fitness(ind, matriz, max_tomas_por_dia) for ind in poblacion]
    return random.choices(poblacion, weights=fitness, k=2)

def crossover(padre1, padre2, tasa_crossover, tomas):
    # cruza 2 padres segun tasa
    if random.random() < tasa_crossover:
        punto_corte = random.randint(1, tomas - 1)
        hijo = padre1[:punto_corte] + padre2[punto_corte:]
        return hijo
    return padre1

def mutar(individuo, tasa_mutacion, tomas, dias):
    # muta aleatoriamente un individuo segun tasa
    if random.random() < tasa_mutacion:
        indice = random.randint(0, tomas - 1)
        individuo[indice] = random.randint(1, dias)
    return individuo

def algoritmo_genetico(matriz, max_tomas_por_dia):
    
    # hiperparametros
    n_poblacion = 100
    generaciones = 100
    tasa_crossover = 0.8
    tasa_mutacion = 0.1
    
    tomas, actores = matriz.shape
    dias = 20  # estimación inicial

    # crea la población
    poblacion = [generar_individuo(tomas, dias) for _ in range(n_poblacion)]

    # aplica los pasos del algoritmo
    for generacion in range(generaciones):
        nueva_poblacion = []
        for _ in range(n_poblacion):
            padre1, padre2 = escoger_padres(poblacion, matriz, max_tomas_por_dia)
            hijo = crossover(padre1, padre2, tasa_crossover, tomas)
            hijo = mutar(hijo, tasa_mutacion, tomas, dias)
            nueva_poblacion.append(hijo)
        poblacion = nueva_poblacion

    # encuentra el mejor individuo al final
    mejor_individuo = max(poblacion, key=lambda ind: calcular_fitness(ind, matriz, max_tomas_por_dia))
    print("Mejor solución encontrada:", mejor_individuo)
    print("Fitness:", calcular_fitness(mejor_individuo, matriz, max_tomas_por_dia))
    print("Días:", len(mejor_individuo))

algoritmo_genetico(matriz, 6)

Mejor solución encontrada: [7, 16, 6, 10, 7, 7, 3, 7, 7, 16, 7, 1, 7, 7, 16, 14, 7, 7, 7, 7, 8, 7, 7, 16, 18, 12, 7, 7, 16, 7]
Fitness: 126
Días: 30


(*)Calcula la complejidad del algoritmo

Respuesta

El cálculo de la complejidad del algoritmo genético no es algo sencillo de calcular porque tiene distintas etapas, y dependerá de la implementación de cada una de estas etapas y la relación entre las variables.

Lo que sabemos es que la complejidad estará en función al tamaño de la población inicial, al número de generaciones, y en este caso, al número de tomas. Adicionalmente, a cada individuo le realizamos una evaluación de fitness, lo seleccionamos, cruzamos y mutamos.

La evaluación del fitness estará en función al número de tomas, ya que necesitamos revisar cada asignación de toma a día en ese individuo. La selección, en base a la población inicial, porque calculamos las probabilidades de selección de los individuos de la población. El crossover requiere copiar hasta el número de tomas para crear un nuevo individuo, y la mutación es la operación más sencilla porque en esta implementación implica cambiar un elemento de la secuencia por individuo.

De esta forma, podemos calcular la complejidad total como una función de la población inicial, el número de generaciones y el número de tomas. En mi implementación, se aproxima a un O(G*P*(T+P+T+1)) = O(G*P*(2T+P) lo cual puede aproximarse a O(G*P*T).

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [34]:
# creo una matriz de 30x10
matriz_random = np.zeros((30,10), int)

# creo 94 1s, al igual que en los datos originales
choices = np.random.choice(matriz_random.size, 94, replace=False)
matriz_random.ravel()[choices] = 1
matriz_random

array([[1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 1, 1, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0,

Aplica el algoritmo al juego de datos generado

Respuesta

In [36]:
# aplico el algoritmo
algoritmo_genetico(matriz, 6)

Mejor solución encontrada: [20, 20, 20, 20, 13, 20, 20, 9, 9, 20, 20, 20, 20, 20, 20, 17, 9, 7, 20, 20, 7, 16, 20, 3, 17, 20, 20, 17, 20, 20]
Fitness: 164
Días: 30


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

1. https://numpy.org/doc/1.26/reference/generated/numpy.zeros.html#numpy.zeros
2. https://numpy.org/doc/1.26/reference/random/index.html#module-numpy.random
3. https://docs.python.org/3/library/random.html
4. https://docs.python.org/3/library/itertools.html
5. https://pandas.pydata.org/docs/reference/api/pandas.Series.to_numpy.html
6. https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/
7. https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6?gi=680aad3684e9
8. https://www.geeksforgeeks.org/brute-force-approach-and-its-pros-and-cons/
9. https://arxiv.org/pdf/1811.04465.pdf

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Creo que, para empezar, es un problema generalizable a otros problemas similares, pensando principalmente por mi experiencia en enrutamiento de vehículos o programación de horarios de personal. En naturaleza es un problema de asignación y evitación de cruces que se repite con frecuencia y amerita seguir siendo estudiado.

En la vida real, existen muchas más restricciones que pueden hacer que el problema sea más complejo como por ejemplo tiempos máximos por sesión, (en el caso de los desplazamientos, relacionandolos al entorno logístico, puede haber un costo de transporte variable), limitaciones de disponibilidad de actores en algunos días, etc.

A medida que crece el problema también es interesante explorar como los algoritmos se comportan a medida que el número de actores, tomas o días aumenta, y puede obligarnos a utilizar otros algoritmos como pueden ser búsqueda tabú, simulated annealing, o un modelo mixto que utilice varios de estos. En un problema similar en mi experiencia laboral, escogimos CPLEX.